# Soil food web interactive analysis and visualization 

This notebook implements our approach to interactive analysis and visualization of soil food webs using Jupyter notebooks and interactive IPython widgets. Therefore, it does not require programming skills beyond setting parameter values and running parts of the notebook. ... BLABLA

## Part 1: preparing the environment and loading the data

In the very first part of the notebook all the required modules and libraries are imported and few helper functions are defined. Next, a data file containing the soil food network in the form of a `NxN` 0/1 matrix is loaded using the widget which allows uploading a file to the memory. Currently, `.csv` files are supported. The matrix is also displayed inline to ensure that the data was loaded correctly.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import importlib
import io
import codecs
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from pyvis.network import Network as net

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual,  HBox, VBox
from IPython.display import display, HTML

from utils.ui_functions import alert, StopExecution, make_download_button
from utils.conf import my_style, my_layout

alert_user = lambda message: display(HTML(alert(message)))

## 1.1 Loading the network matrix

In [3]:
widgets.FileUpload(accept='.csv',multiple=False)
matrix_uploader = widgets.FileUpload()
display(matrix_uploader)

FileUpload(value={}, description='Upload')

In [4]:
try:
    matrix_file_name, matrix_file_data = list(matrix_uploader.value.items())[0]
except Exception:
    alert_user('Please load the food web matrix data file the cell above!\nThe program cannot continue without this data.')
    raise StopExecution    
sfw_matrix = pd.read_csv(io.StringIO(codecs.decode(matrix_file_data['content'], encoding="utf-8")),
                         header=0, 
                         index_col=0, 
                         sep=";")

if sfw_matrix.shape[0]!=sfw_matrix.shape[1]:
    alert_user('Invalid graph matrix: it must have NxN shape.')
    raise StopExecution

@interact(nrows=widgets.IntSlider(description='Rows:', min=3, max=sfw_matrix.shape[1], step=1, value=5),
          ncols=widgets.IntSlider(description='Columns:', min=3, max=sfw_matrix.shape[0], step=1, value=10))
def show_soil_food_web_matrix(nrows, ncols):
    pd.set_option("display.max_rows", nrows)
    pd.set_option("display.max_columns", ncols)
    display(sfw_matrix)

interactive(children=(IntSlider(value=5, description='Rows:', max=24, min=3), IntSlider(value=10, description=…

### 1.2 Loading the trophic level matrix

If we have the trophic level data it is possible to load it here. It must have the `Nx1` shape. If no such data is available, the following two cells are skipped and the trophic level will not be used (all nodes will have the same value).

In [5]:
widgets.FileUpload(accept='csv',multiple=False)
trophic_level_uploader = widgets.FileUpload()
display(trophic_level_uploader)

FileUpload(value={}, description='Upload')

In [6]:
try:
    trophic_level_file_name, trophic_level_file_data = list(trophic_level_uploader.value.items())[0]
except Exception:
    alert_user('Please load the file with trophic level data in the cell above!\nThe program cannot continue without trophic level data.')
    raise StopExecution
trophic_levels = pd.read_csv(io.StringIO(codecs.decode(trophic_level_file_data['content'], encoding='utf-8')),
                             index_col=0, header=None, delimiter=";", names=['trophic_level'])

if trophic_levels.shape[1]!=1:
    alert_user('Invalid thropic matrix: it must have Nx1 shape.')
    raise StopExecution
if trophic_levels.shape[0] != sfw_matrix.shape[0]:
    alert_user('Invalid number of rows in the trophic matrix: trophic level must be defined for all organisms (nodes).')
    raise StopExecution
    
pd.set_option('display.max_rows', None)
display(trophic_levels.sort_values('trophic_level'))
trophic_levels = trophic_levels.squeeze()
trophic_levels_dict = trophic_levels.to_dict()

,trophic_level
49 Primary (heterotrophic) producer fungi,0
48 Primary (heterotrophic) producer bacteria,0
#0 Plant autotrophs (roots),0
11 Plant-feeding nematode,1
22 Microphytophage mite (feeding on fungi),1
"41 ""Passive lifestage, substrate-related nematode""",1
37 Bacterivore flagellates,1
36 Bacterivore amoebae,1
12 Macrophytophage and panphytophage mite,1
34 Bacterivore enchytraeid,1


## Part 2

### 2.1 Network construction

Using the provided adjacency matrix data and the trophic level data we will create the network structure and offer basic editing functions. 

In [7]:
G = nx.from_pandas_adjacency(sfw_matrix, create_using=nx.DiGraph)
# remove default edge weights
for n1, n2, d in G.edges(data=True):
    d.clear()
    
graph_name_input = widgets.Text(placeholder='The name of this network', value='My network', description='Name:', disabled=False, style=my_style)
rename_button = widgets.Button(description='Rename', disabled=False, button_style='primary',icon='edit')
rename_log = widgets.Output(layout={'border': '1px solid blue'})
display(rename_log)
def rename_graph(widget_name):
    G.name = graph_name_input.value
    rename_log.clear_output()
    rename_log.append_display_data(f'Graph renamed to "{G.name}"')
rename_button.on_click(rename_graph)

# TODO make sure the network has a name
HBox([graph_name_input, rename_button])

Output(layout=Layout(border='1px solid blue'))

In [8]:
if not G.name:
    alert_user('The network does not have a name!\nPlease use the "Rename" button in the above cell.')
    raise StopExecution

print('--- Basic network info ---')
print(f'Name: {G.name}')
typ = 'directed' if nx.is_directed(G) else 'undirected'
print(f'Type: {typ}')
print(f'Nodes: {G.number_of_nodes()}')
print(f'Edges: {G.number_of_edges()}')
sc = 'yes' if nx.is_strongly_connected(G) else 'no'
print(f'Strongly connected: {sc}')
print(f'Density: {nx.density(G):.5f}')

--- Basic network info ---
Name: My network
Type: directed
Nodes: 24
Edges: 94
Strongly connected: no
Density: 0.17029


### 2.2 Network editing

In [9]:
select_node_dropdown = widgets.Dropdown(options=sorted(list(G.nodes)), description='Selected node:', disabled=False, 
                                        layout=my_layout, style=my_style)
delete_button = widgets.Button(description='Delete this node', disabled=False, button_style='danger',icon='trash')
delete_log = widgets.Output(layout={'border': '1px solid blue'})
def button_action(widget_name):
    selected_node = select_node_dropdown.value
    if selected_node:
        G.remove_node(selected_node)
        select_node_dropdown.options = sorted(list(G.nodes))
        delete_log.append_display_data(f'Node "{selected_node}" was removed')
delete_button.on_click(button_action)
display(delete_log)
HBox([select_node_dropdown, delete_button])

Output(layout=Layout(border='1px solid blue'))

# Part 3: network visualization

In [13]:
import textwrap
import webbrowser
from pyvis.network import Network
from utils import conf
from utils import functions
from utils.ui_functions import make_download_button
import copy

out = widgets.Output(layout={'border': '0px'})
@out.capture()
def draw_network_inline(widget_name):
    out.clear_output()
    vis_G = prepare_visualization()
    GG = copy.copy(vis_G)
    GG.conf = False
    GG.set_options(conf.hierarchical_food_web_visual_settings_inline)
    display(HTML(GG.generate_html()))
    # display(GG.show(f'{G.name}.html'))

def draw_network_new_tab(widget_name):
    vis_G = prepare_visualization()
    GG = copy.copy(vis_G)
    GG.conf = True
    GG.set_options(conf.hierarchical_food_web_visual_settings_standalone)
    web_page = f'{G.name}.html'
    GG.write_html(web_page)
    webbrowser.open(web_page)

color_select = widgets.Dropdown(options=['viridis', 'plasma', 'inferno', 'magma', 'cividis', 'bone', 'pink',
                                         'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia', 'hot', 'afmhot', 
                                         'gist_heat', 'copper'],
                                value='spring',           
                                description='Colormap:',
                                disabled=False)
shape_select = widgets.Dropdown(options=['ellipse', 'circle', 'database', 'box', 'text', 'diamond', 'dot', 'star', 
                                         'triangle', 'triangleDown', 'hexagon', 'square'],
                                value='box',           
                                description='Node shape:',
                                disabled=False)
label_width_slider  = widgets.IntSlider(value=20, min=6, max=40, step=2, description='Label width:', disabled=False,
                                        continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
draw_inline_button = widgets.Button(description='Draw inline', disabled=False, button_style='primary')
draw_outside_button = widgets.Button(description='Open in new tab', disabled=False, button_style='primary')
draw_outside_button.style.button_color = 'green'
draw_inline_button.on_click(draw_network_inline)
draw_outside_button.on_click(draw_network_new_tab)

def prepare_visualization():
    colors = functions.get_colors(len(set(trophic_levels)), colormap=color_select.value)
    vis_G = Network('800px', '1200px', notebook=False)
    # add nodes
    for node in G.nodes():
        vis_G.add_node(node, 
                       label='\n'.join(textwrap.wrap(node, label_width_slider.value, break_long_words=True)), 
                       level=max(trophic_levels) - trophic_levels_dict[node], 
                       shape=shape_select.value,
                       color=colors[trophic_levels_dict[node]])
    # and edges
    for source, target in list(G.edges()):
        vis_G.add_edge(source, target)
    return vis_G  


display(VBox([color_select, shape_select, label_width_slider, draw_inline_button, draw_outside_button]))
display(make_download_button(G.name))
display(out)

HTML(value='<html>\n<head>\n<meta name="viewport" content="width=device-width, initial-scale=1">\n</head>\n<bo…

Output(layout=Layout(border='0px'))